# Job Dataset Exploration

In [1]:
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

In [33]:
user_ids = [1, 2, 3, 1, 2, 3, 1]
ratings = [1, 1, 1, 1, 1, 1, 1]
titles = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

user_ids2 = [1, 3, 2, 3, 2, 1, 1, 3, 2, 3, 2, 1]
ratings2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
titles2 = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']

data = {"userId": user_ids, "title": titles, "ratings": ratings}
df = pd.DataFrame(data)
df = df.sort_values(by = ['userId'])

data2 = {"userId": user_ids2, "title": titles2, "ratings": ratings2}
df2 = pd.DataFrame(data2)
df2 = df2.sort_values(by = ['userId'])

display(df)
display(df2)

,userId,title,ratings
0,1,a,1
3,1,d,1
6,1,g,1
1,2,b,1
4,2,e,1
2,3,c,1
5,3,f,1


,userId,title,ratings
0,1,a,0
5,1,f,0
6,1,g,0
11,1,l,0
2,2,c,0
4,2,e,0
8,2,i,0
10,2,k,0
1,3,b,0
3,3,d,0


In [52]:
df_joined = pd.concat([df, df2])
df_joined = df_joined.sort_values(by = ['ratings'], ascending=False)
display(df_joined)

df_dropped = df_joined.drop_duplicates(subset=['title'], keep='first')
df_dropped = df_dropped.sort_values(by = ['userId'])
df_dropped = df_dropped.reset_index().drop(columns=['index'])
display(df_dropped)

,userId,title,ratings
0,1,a,1
6,1,g,1
1,2,b,1
4,2,e,1
2,3,c,1
5,3,f,1
3,1,d,1
8,2,i,0
7,3,h,0
3,3,d,0


,userId,title,ratings
0,1,a,1
1,1,g,1
2,1,d,1
3,1,l,0
4,2,b,1
5,2,e,1
6,2,i,0
7,2,k,0
8,3,c,1
9,3,f,1


In [6]:
# read the data (about applications)
data = pd.read_csv('dataset/apps.tsv', delimiter='\t')
#display(data.head(10)) # check data being read properly

# extract relevant data
df_ui = data[["UserID", "JobID"]]
#display(df_ui.head(10)) # check data being read properly

# construct dataframe in format (user, item, rating) via column addition
df_ui.insert(2, "rating", np.full((df_ui.shape[0], 1), 1, float))
df_ui = df_ui.rename(columns={"UserID": "user", "JobID": "item"})
display(df_ui.head(10)) # check data being read properly


,user,item,rating
0,47,169528,1.0
1,47,284009,1.0
2,47,2121,1.0
3,47,848187,1.0
4,47,733748,1.0
5,47,576958,1.0
6,47,262470,1.0
7,47,602298,1.0
8,72,834662,1.0
9,72,1020903,1.0


In [ ]:

# Compute nearest neighbours
userID = 203
itemID = 602298
ratings = df_ui[df_ui['user'] == userID]
ratings2 = ratings[['item', 'rating']]
ratings2 = ratings2.set_index('item')


# We use the collaborative user algorithm UserUser, that use the nearest neighbors 
num_recs = 10  # Number of recommendations to generate
user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(ratings)

selected_movies_useruser = recsys.recommend(203, 10) # generate 10 recommendations for the selected user 